In [1]:
import time
import asyncio
import requests
import pymongo
import json
from bson.json_util import dumps
from pymongo import MongoClient
from indy import anoncreds, did, ledger, pool, wallet, IndyError

async def run():
    print("Getting started -> Getting Verinym")

    # Set protocol version 2 to work with Indy Node 1.4
    await pool.set_protocol_version(2)

    # pool_ = {
    #     'name': 'pool1',
    #     'config': json.dumps({"genesis_txn": '/home/indy/sandbox/pool_transactions_genesis'})
    # }
    # print("Open Pool Ledger: {}".format(pool_['name']))

    # try:
    #     await pool.create_pool_ledger_config(pool_['name'], pool_['config'])
    # except IndyError as ex:
    #     if ex.error_code == 306:
    #         pass
    # pool_['handle'] = await pool.open_pool_ledger(pool_['name'], None)

    client = MongoClient(host='steward_db', port=27017, username='root', password='pass', authSource="admin")
    db = client["steward_db"]

    #pool_ = db.steward_tb.find_one({"name": "pool1"})
    #pool_['handle'] = await pool.open_pool_ledger(pool_['name'], None)

    response = requests.get('http://steward:5000/get_pool_config')
    print(response)
    pool_ = response.json()
    print(pool_)
    print(pool_['config'])

    verinym_for = db.steward_tb.find_one({"name": "verinym_for"})

    print ("---------*****------")
    print (verinym_for)

    entity = db.steward_tb.find_one({"name": verinym_for["value"]})

    steward = db.steward_tb.find_one({"name": "Sovrin Steward"})

    #db.steward_tb.find({},{_id:0});

    print(entity)
    print(steward)

    await getting_verinym(steward, entity)

    db.steward_tb.find_one_and_update({"name": verinym_for["value"]}, { "$set": entity }, upsert=True);

    db.steward_tb.find_one_and_update({"name": "Sovrin Steward"}, { "$set": steward }, upsert=True);

    print("==============")

    print(entity)

async def getting_verinym(from_, to):
    from_['info'] = {
        'did': to['did'],
        'verkey': to['key'],
        'role': to['role'] or None
    }
    print(from_['wallet'])
    await send_nym(from_['pool'], from_['wallet'], from_['did'], from_['info']['did'],
                   from_['info']['verkey'], from_['info']['role'])

async def send_nym(pool_handle, wallet_handle, _did, new_did, new_key, role):
    nym_request = await ledger.build_nym_request(_did, new_did, new_key, None, role)
    await ledger.sign_and_submit_request(pool_handle, wallet_handle, _did, nym_request)


if __name__ == '__main__':
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    await run()
    time.sleep(1)  # FIXME waiting for libindy thread complete


Getting started -> Getting Verinym
<Response [200]>
{'_id': {'$oid': '6268028b85d0228b8f9696aa'}, 'name': 'pool1', 'config': '{"genesis_txn": "/home/indy/sandbox/pool_transactions_genesis"}', 'handle': 2}
{"genesis_txn": "/home/indy/sandbox/pool_transactions_genesis"}
---------*****------
{'_id': ObjectId('626803447d1cfe49fac328fa'), 'name': 'verinym_for', 'value': 'Supplier'}
{'_id': ObjectId('62680344fb2455aaf97a6b8a'), 'name': 'Supplier', 'wallet_config': {'id': 'supplier_wallet'}, 'wallet_credentials': {'key': 'supplier_wallet_key'}, 'pool': 2, 'role': 'TRUST_ANCHOR', 'wallet': 3, 'did': 'QeUxxCBYJhUBzccDSkCg4t', 'key': 'DtWFm3rcComZpENMkY3uZjMHs2kxodYv4rihj1aFBuU7'}
{'_id': ObjectId('6268029d85d0228b8f9696ab'), 'name': 'Sovrin Steward', 'wallet_config': '{"id": "sovrin_steward_wallet"}', 'wallet_credentials': '{"key": "steward_wallet_key"}', 'pool': 2, 'seed': '000000000000000000000000Steward1', 'wallet': 5, 'did_info': '{"seed": "000000000000000000000000Steward1"}', 'did': 'Th7Mp

WalletInvalidHandle: 